In [28]:
from gensim.models import word2vec 

C:\Users\pranith\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
import spacy
spacy.load("en_core_web_sm")
from spacy.lang.en import English
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [7]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pranith\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [9]:
for w in ['dogs', 'ran', 'discouraged']:
    print(w, get_lemma(w), get_lemma2(w))

dogs dog dog
ran run ran
discouraged discourage discouraged


In [10]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pranith\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [13]:
import random
text_data = []
with open('dataset.csv') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            print(tokens)
            text_data.append(tokens)

['predict', 'reflectance', 'function', 'complex', 'surface']
['analysis', 'signal', 'power', 'integrity', 'using', 'multilayered', 'finite', 'difference', 'method']
['custom', 'design', 'subbyte', 'module', 'signal', 'independent', 'power', 'consumption']
['sup>2</sup', 'parallel', 'pipelined', 'efficient', 'architecture', 'discrete', 'wavelet', 'transform']
['comparative', 'evaluation', 'javascript', 'framework']
['toward', 'simple', 'criterion', 'establish', 'capacity', 'scaling', 'wireless', 'network']
['improve', 'class', 'buffer', 'base', 'differential', 'flip', 'voltage', 'followers']
['measurement', 'base', 'editing', 'diffuse', 'albedo', 'consistent', 'interreflection']
['analysis', 'blocking', 'probability', 'noise', 'crosstalk', 'impair', 'optical', 'network']
['optimal', 'allocation', 'scalable', 'video', 'multicast', 'wimax']
['adapt', 'dynamic', 'service', 'composition']
['base', 'traffic', 'localization']
['digital', 'halftoning', 'space', 'filling', 'curve']
['rapid', 'p

In [14]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)

In [15]:
corpus = [dictionary.doc2bow(text) for text in text_data]

In [16]:
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [17]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')

In [18]:
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.041*"base" + 0.041*"comparative" + 0.022*"technology" + 0.022*"module"')
(1, '0.030*"power" + 0.030*"method" + 0.030*"signal" + 0.030*"analysis"')
(2, '0.041*"base" + 0.022*"design" + 0.022*"combining" + 0.022*"rapid"')
(3, '0.028*"class" + 0.028*"hearing" + 0.028*"micropower" + 0.028*"efficiency"')
(4, '0.041*"network" + 0.022*"crosstalk" + 0.022*"toward" + 0.022*"flexible"')


In [19]:
new_doc = 'Practical Bayesian Optimization of Machine Learning Algorithms'
new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))

[]
[(0, 0.2), (1, 0.2), (2, 0.2), (3, 0.2), (4, 0.2)]


In [20]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 3, id2word=dictionary, passes=15)
ldamodel.save('model3.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)


(0, '0.029*"base" + 0.016*"comparative" + 0.016*"power" + 0.016*"class"')
(1, '0.028*"architecture" + 0.028*"technology" + 0.028*"base" + 0.016*"design"')
(2, '0.030*"network" + 0.030*"analysis" + 0.017*"method" + 0.017*"power"')


In [21]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=15)
ldamodel.save('model10.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.029*"method" + 0.029*"signal" + 0.029*"analysis" + 0.029*"sheet"')
(1, '0.082*"technology" + 0.082*"lighting" + 0.007*"base" + 0.007*"power"')
(2, '0.060*"base" + 0.060*"diffuse" + 0.060*"albedo" + 0.060*"measurement"')
(3, '0.040*"design" + 0.040*"domain" + 0.040*"language" + 0.040*"model"')
(4, '0.040*"power" + 0.040*"efficiency" + 0.040*"video" + 0.040*"amplifier"')
(5, '0.067*"digital" + 0.067*"filling" + 0.067*"space" + 0.067*"halftoning"')
(6, '0.071*"comparative" + 0.071*"javascript" + 0.071*"framework" + 0.071*"evaluation"')
(7, '0.040*"architecture" + 0.040*"transform" + 0.040*"criterion" + 0.040*"parallel"')
(8, '0.057*"base" + 0.057*"class" + 0.057*"voltage" + 0.057*"followers"')
(9, '0.051*"base" + 0.051*"network" + 0.027*"crosstalk" + 0.027*"optical"')


In [22]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

In [26]:
import pyLDAvis.gensim_models
lda_display = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

C:\Users\pranith\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\pranith\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
